* Load and display vehicle images
* Threshold saliency map
* Use salient object detection to create an alpha mask
* Use the alpha, original image, and background

In [ ]:
import cv2, imageio, os, warnings
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import matplotlib.gridspec as gridspec

from types import SimpleNamespace
from scipy import misc

%matplotlib inline
warnings.filterwarnings('ignore')

In [ ]:
def make_saliency_map(car_name):
    # Read the file into memory
    filename = './images/original/{0}.jpeg'.format(car_name)
    image = cv2.imread(filename)
    
    # Define the Saliency Fine Grained
    saliency_fine = cv2.saliency.StaticSaliencyFineGrained_create()
    (success, saliency_map_fine) = saliency_fine.computeSaliency(image)

    # Create a threshold from the saliency
    threshMap_fine = cv2.threshold(saliency_map_fine, 0, 255,cv2.THRESH_BINARY | cv2.THRESH_OTSU)[1]
    
    # Save the saliency map
    out_folder = './images/saliency'
    out_filename = '{0}.png'.format(car_name)
    out_filename_thresh = '{0}_thresh.png'.format(car_name)
    
    # write the file to disk
    imageio.imwrite(os.path.join(out_folder,out_filename),saliency_map_fine)
    imageio.imwrite(os.path.join(out_folder,out_filename_thresh),threshMap_fine)

In [ ]:
def make_saliency_cmap(car_name):
    filename = './images/original/{0}.jpeg'.format(car_name)
    cmaps = ['viridis', 'plasma', 'inferno', 'magma', 'seismic', 'spectral']
    for maps in cmaps:
        dpi = 80

        im_data = plt.imread(filename)
        height, width, nbands = im_data.shape

        # What size does the figure need to be in inches to fit the image?
        figsize = width / float(dpi), height / float(dpi)

        # Create a figure of the right size with one axes that takes up the full figure
        fig = plt.figure(figsize=figsize)
        ax = fig.add_axes([0, 0, 1, 1])

        # Hide spines, ticks, etc.
        ax.axis('off')

        image = cv2.imread(filename)

        saliency = cv2.saliency.StaticSaliencyFineGrained_create()
        (success, saliencyMap) = saliency.computeSaliency(image)
        
        ax.imshow(saliencyMap, cmap=maps, interpolation="bilinear")
        ax.set(xlim=[0, width], ylim=[height, 0], aspect=1)
        filename = './images/saliency_cmap/{0}_{1}.png'.format(car_name,maps)
        fig.savefig(filename, dpi=dpi, transparent=True)
        
        
        threshMap = cv2.threshold(saliencyMap, 0, 255,cv2.THRESH_BINARY | cv2.THRESH_OTSU)[1]
        ax.imshow(threshMap, cmap=maps, interpolation="bilinear")
        ax.set(xlim=[0, width], ylim=[height, 0], aspect=1)
        filename = './images/saliency_cmap/{0}_{1}_threshold.png'.format(car_name,maps)
        fig.savefig(filename, dpi=dpi, transparent=True)

In [ ]:
# Salient Object Detection - Alpha Mask
g_mean = np.array(([126.88,120.24,112.19])).reshape([1,1,3])
OUTPUT = "./images/alpha"
def rgba2rgb(img):
    return img[:,:,:3]*np.expand_dims(img[:,:,3],2)
def make_saliency_alpha(args):
    with tf.Session(config=tf.ConfigProto()) as sess:
        saver = tf.train.import_meta_graph('./salient/graph/my-model.meta')
        saver.restore(sess,tf.train.latest_checkpoint('./salient/model'))
        image_batch = tf.get_collection('image_batch')[0]
        pred_mattes = tf.get_collection('mask')[0]

        rgb = imageio.imread(args.rgb)

        if rgb.shape[2]==4:
            rgb = rgba2rgb(rgb)
        origin_shape = rgb.shape[:2]
        rgb = np.expand_dims(misc.imresize(rgb.astype(np.uint8),[320,320,3], interp="bilinear").astype(np.float32)-g_mean,0)

        feed_dict = {image_batch:rgb}
        pred_alpha = sess.run(pred_mattes,feed_dict = feed_dict)
        final_alpha = misc.imresize(np.squeeze(pred_alpha),origin_shape)
        final_name = '{0}_{1}.png'.format(args.car, args.cmap)
        imageio.imwrite(os.path.join(OUTPUT,final_name),final_alpha)

In [ ]:
def make_image(car_name, cmap):
    # Read the images
    foreground = cv2.imread("./images/original/{0}.jpeg".format(car_name))
    background = cv2.imread("./images/background.png")
    alpha = cv2.imread("./images/alpha/{0}_{1}.png".format(car_name, cmap))

    # Convert uint8 to float
    foreground = foreground.astype(float)
    background = background.astype(float)

    # Normalize the alpha mask to keep intensity between 0 and 1
    alpha = alpha.astype(float)/255

    # Multiply the foreground with the alpha matte
    foreground = cv2.multiply(alpha, foreground)

    # Multiply the background with ( 1 - alpha )
    background = cv2.multiply(1.0 - alpha, background)

    # Add the masked foreground and background.
    outImage = cv2.add(foreground, background)
    
    out_folder = './images/generated'
    out_filename = '{0}_{1}.png'.format(car_name, cmap)
    imageio.imwrite(os.path.join(out_folder,out_filename),outImage)

In [ ]:
# GLOBAL
CAR = '1C4PJLCX8KD22313104'
make_saliency_map(CAR)
make_saliency_cmap(CAR)

# Create the alpha mask
INPUT = "./images/original/{0}.jpeg".format(CAR)
d = {'rgb': INPUT, 'car':CAR, 'cmap':'Original'}
n = SimpleNamespace(**d)
make_saliency_alpha(n)

cmaps = ['viridis', 'plasma', 'inferno', 'magma', 'seismic', 'spectral']
for maps in cmaps:
    cmaps = "./images/saliency_cmap/{0}_{1}.png".format(CAR, maps)
    d = {'rgb': cmaps, 'car':CAR, 'cmap':maps}
    n = SimpleNamespace(**d)
    make_saliency_alpha(n)

# Join the alpha mask with the original
cmaps_join = ['Original', 'viridis', 'plasma', 'inferno', 'magma', 'seismic', 'spectral']
for c in cmaps_join:
    make_image(CAR, c)


## Saliency map
* Initialize OpenCV's static fine grained saliency detector and compute the saliency map
* In order to process for contours with a binary map, compute convex hull's, extract bounding boxes, add threshold the saliency map

In [ ]:
image = cv2.imread('./images/original/{0}.jpeg'.format(CAR))
saliency = cv2.saliency.StaticSaliencySpectralResidual_create()
(success, saliencyMap) = saliency.computeSaliency(image)
saliencyMapSpectral = (saliencyMap * 255).astype("uint8")

saliency = cv2.saliency.StaticSaliencyFineGrained_create()
(success, saliencyMap) = saliency.computeSaliency(image)

threshMap = cv2.threshold(saliencyMap, 0, 255,cv2.THRESH_BINARY | cv2.THRESH_OTSU)[1]

gs = gridspec.GridSpec(2, 2)
gs.update(wspace = 0.10, hspace = 0.10)

fig, ax = plt.subplots(figsize = (16, 16))

plt.subplot(gs[0])
plt.imshow(image, cmap = 'gray')
plt.title('Original')
plt.subplot(gs[1])
plt.imshow(saliencyMap)
plt.title('saliencyMap')
plt.subplot(gs[2])
plt.imshow(threshMap, cmap = 'gray')
plt.title('Thresh')
plt.subplot(gs[3])
plt.imshow(saliencyMapSpectral, cmap = 'gray')
plt.title('SaliencySpectralResidual')
